# Privacy Audit Training

使用 LoRA 微调 Qwen2.5-0.5B-Instruct 模型

## 1. 安装依赖

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets transformers peft trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 16.3 MB/s eta 0:00:00


## 2. 检查 GPU

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
GPU Memory: 85.2 GB


## 3. 上传数据文件

请上传 `wiki_trimmed_with_canary.jsonl` 文件到 Colab

## 4. 训练代码

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

# ----------------------------------
# 模型与数据设置
# ----------------------------------
model_name = "Qwen/Qwen2.5-0.5B-Instruct"  # 从 HuggingFace 下载
train_data_file = "/content/drive/MyDrive/PrivacyAudit/wiki_trimmed_with_canary.jsonl"
output_dir = "qwen2_0p5b_sft_A100"

# ----------------------------------
# 1) 加载 Tokenizer 与 Model
# ----------------------------------
print("[INFO] Loading tokenizer and base model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"[OK] Tokenizer loaded. Vocab size: {len(tokenizer)}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
print("[OK] Model loaded successfully!")

# ----------------------------------
# 2) 加载训练集
# ----------------------------------
print("[INFO] Loading training dataset...")
train_dataset = load_dataset("json", data_files=train_data_file, split="train")
print(f"[OK] Dataset loaded. Number of examples: {len(train_dataset)}")

# ----------------------------------
# 3) PEFT/LoRA 配置
# ----------------------------------
print("[INFO] Configuring LoRA/PEFT...")
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("[OK] LoRA configuration applied!")
model.print_trainable_parameters()

# ----------------------------------
# 4) SFT 训练设置 (T4 GPU 优化)
# ----------------------------------
print("[INFO] Setting up SFT Trainer...")
training_args = SFTConfig(
    learning_rate=2e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    output_dir=output_dir,
    logging_steps=50,
    save_steps=200,
    bf16=True,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)
print("[OK] Trainer initialized successfully!")

[INFO] Loading tokenizer and base model...
[OK] Tokenizer loaded. Vocab size: 151665
[OK] Model loaded successfully!
[INFO] Loading training dataset...


Generating train split: 0 examples [00:00, ? examples/s]

[OK] Dataset loaded. Number of examples: 10010
[INFO] Configuring LoRA/PEFT...
[OK] LoRA configuration applied!
trainable params: 2,162,688 || all params: 496,195,456 || trainable%: 0.4359
[INFO] Setting up SFT Trainer...


Adding EOS to train dataset:   0%|          | 0/10010 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10010 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10010 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


[OK] Trainer initialized successfully!


## 5. 开始训练

In [ ]:
print("=" * 60)
print("[INFO] Starting fine-tuning...")
print("=" * 60)
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


[INFO] Starting fine-tuning...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maxuebin (maxuebin-verimind) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,2.501100
100,2.397700
150,2.361600
200,2.408700
250,2.375400
300,2.354200


TrainOutput(global_step=313, training_loss=2.3988038389065776, metrics={'train_runtime': 654.7992, 'train_samples_per_second': 15.287, 'train_steps_per_second': 0.478, 'total_flos': 1.78985834814336e+16, 'train_loss': 2.3988038389065776})

## 6. 保存模型

In [ ]:
print("[INFO] Saving trained model...")
trainer.save_model(output_dir)
print(f"[DONE] Model saved to {output_dir}")

[INFO] Saving trained model...
[DONE] Model saved to qwen2_0p5b_sft_A100


## 7. 下载训练好的模型

In [ ]:
# 打包并下载
!zip -r qwen2_0p5b_sft_A100.zip qwen2_0p5b_sft_A100/
files.download('qwen2_0p5b_sft_A100.zip')

	zip warning: name not matched: qwen2_0p5b_sft_A100/

zip error: Nothing to do! (try: zip -r qwen2_0p5b_sft_A100.zip . -i qwen2_0p5b_sft_A100/)


NameError: name 'files' is not defined